In [ ]:

import csv
import codecs
import pandas as pd                          # package to store and manipulate data
import numpy as np                           # package to store and manipulate data
import matplotlib.pyplot as plt              # plotting package
import seaborn as sns                        # plotting package 
import sklearn                               # model building package
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import re                                    # packages to clean text
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
import nltk as nlp
nltk.download('wordnet')
import string
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df= pd.read_csv('/troll_tweets_for_bert_sentiment.csv', encoding = "unicode_escape")

In [ ]:
df.head()

,tweet_created_at,tweet_text,sentiment
0,2020-09-30 21:25:00,RT AlexaZtoA More money spent on adoration of ...,pos
1,2020-09-30 21:20:00,Drallasta httpstcoP9Tv7JD0hT,neu
2,2020-09-30 21:14:00,osbornetravel guyverhofstadt NathalieLoiseau v...,pos
3,2020-09-30 21:12:00,KatyMontgomerie Canada the home of acceptance ...,neg
4,2020-09-30 21:11:00,KatyMontgomerie Even some Canadians are horrif...,neg


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df["sentiment"] = ord_enc.fit_transform(df[["sentiment"]])
df[["sentiment"]].head(11)

,sentiment
0,2.0
1,1.0
2,2.0
3,0.0
4,0.0
5,0.0
6,0.0
7,1.0
8,1.0
9,1.0


In [ ]:
dataframe = df[["tweet_text", "sentiment"]]

In [ ]:
dataframe

,tweet_text,sentiment
0,RT AlexaZtoA More money spent on adoration of ...,2.0
1,Drallasta httpstcoP9Tv7JD0hT,1.0
2,osbornetravel guyverhofstadt NathalieLoiseau v...,2.0
3,KatyMontgomerie Canada the home of acceptance ...,0.0
4,KatyMontgomerie Even some Canadians are horrif...,0.0
...,...,...
18288,RT ProjectLincoln We celebrate our freedom tom...,2.0
18289,page88 Congrats I celebrated 14 last week Sobr...,2.0
18290,RT ProjectLincoln \n \n In this\n house\n ...,1.0
18291,RT AngrierWHStaff,1.0


In [ ]:
dataframe["sentiment"] = dataframe.sentiment.astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
dataframe

,tweet_text,sentiment
0,RT AlexaZtoA More money spent on adoration of ...,2
1,Drallasta httpstcoP9Tv7JD0hT,1
2,osbornetravel guyverhofstadt NathalieLoiseau v...,2
3,KatyMontgomerie Canada the home of acceptance ...,0
4,KatyMontgomerie Even some Canadians are horrif...,0
...,...,...
18288,RT ProjectLincoln We celebrate our freedom tom...,2
18289,page88 Congrats I celebrated 14 last week Sobr...,2
18290,RT ProjectLincoln \n \n In this\n house\n ...,1
18291,RT AngrierWHStaff,1


In [ ]:
seq_len = 64
num_samples = len(df)

Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))
Xids.shape

(18293, 64)

In [ ]:
!pip install transformers

In [ ]:
import transformers

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
for i, tweet_text in enumerate(df['tweet_text']):
  tokens= tokenizer.encode_plus(str(tweet_text), max_length=seq_len, truncation= True, 
                                padding = 'max_length', add_special_tokens = True, return_tensors = 'tf')
  Xids[i, :]= tokens['input_ids']
  Xmask[i, :]= tokens['attention_mask']

In [ ]:
Xids

array([[ 101.,  155., 1942., ...,    0.,    0.,    0.],
       [ 101., 1987., 5727., ...,    0.,    0.,    0.],
       [ 101.,  184., 1116., ...,    0.,    0.,    0.],
       ...,
       [ 101.,  155., 1942., ...,    0.,    0.,    0.],
       [ 101.,  155., 1942., ...,    0.,    0.,    0.],
       [ 101.,  155., 1942., ...,    0.,    0.,    0.]])

In [ ]:
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [ ]:
arr =dataframe['sentiment'].values

In [ ]:
arr

array([2, 1, 2, ..., 1, 1, 1])

In [ ]:
labels = np.zeros((num_samples, arr.max()+1))

In [ ]:
labels.shape

(18293, 3)

In [ ]:
labels[np.arange(num_samples), arr] =1

In [ ]:
labels

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [ ]:
import tensorflow as tf

In [ ]:
dataset =tf.data.Dataset.from_tensor_slices((Xids, Xmask,labels))
dataset.take(1)

<TakeDataset shapes: ((64,), (64,), (3,)), types: (tf.float64, tf.float64, tf.float64)>

In [ ]:
Xids[0, :].shape

(64,)

In [ ]:
def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask' : masks}, labels

In [ ]:
dataset = dataset.map(map_func)

In [ ]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (64,), attention_mask: (64,)}, (3,)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [ ]:
batch_size = 128
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (128, 64), attention_mask: (128, 64)}, (128, 3)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.float64)>

In [ ]:
split = 0.8
size = int(((num_samples/batch_size)*split))

In [ ]:
size

114

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)
del dataset

In [ ]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
bert.summary()

Model: "tf_bert_model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
input_ids  = tf.keras.layers.Input(shape= (seq_len,), name= 'input_ids', dtype='int32')
mask  = tf.keras.layers.Input(shape= (seq_len,), name= 'attention_mask', dtype='int32')
embeddings = bert.bert(input_ids, attention_mask = mask)[1]
x = tf.keras.layers.Dense(512, activation  ='relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation='softmax', name ='outputs')(x)

In [ ]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs = y)
#model.layers[2].trainable =False
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 64)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 512)          393728      bert[1][1]                 

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr= 1e-5, decay = 1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy()


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
model.compile(optimizer=optimizer, loss =loss, metrics=[acc])

In [ ]:
history = model.fit(train_ds, epochs=1)

114/114 [==============================] - 176s 1s/step - loss: 0.6087 - categorical_accuracy: 0.7606


In [ ]:
results = model.evaluate(val_ds)

28/28 [==============================] - 16s 494ms/step - loss: 0.4596 - categorical_accuracy: 0.8214
